In [1]:
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import adjusted_mutual_info_score
from math import floor, ceil
from tqdm import tqdm
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data.dataset import Dataset, TensorDataset
from torch.utils.data.dataloader import DataLoader

In [ ]:
################################## Config Setup ####################################
dataset = 'Dataset'
config = {'min_recall_value': 0.5, 'AMI_correlation_threshold': 0.125, 'batch_size': 32, 'dropout': 0, 'layer_size': 128, 'num_layers': 3, 'lr': 0.0001, 'epochs': 9}

# Size of the coarse position prior as a number of images
coarse_prior_size = 75
# The nominated minimum performance target as a decimal representing recall@1
target_min_recall = config['min_recall_value']
# The step size to take between coarse position priors
step_size = 15

# Percentage of data to train (both training and validation)
train_percent = 0.5
# Percentage of data assigned to each split (train, validation, test)
split_vals = [0.3, 0.2, 0.5]

############################## Load Previously Computed Data #########################
# This data is computed previously in another piece of the code (if not uploaded yet, it will be shortly)
coarse_prior_appearance_var = np.load('Path_To_Data')
min_seq_len_for_target = np.load('Path_To_Data')
min_seq_len_for_target = min_seq_len_for_target.squeeze()

# This is just finding the indices to use for the data splits
cutoff = int(train_percent*coarse_prior_appearance_var.shape[0])
train =  int((train_percent-0.2)*coarse_prior_appearance_var.shape[0])
valid = cutoff-train

In [4]:
################################## Definitions for the Model and Loss Term #################################
class Basic_NN(nn.Module):
    def __init__(self, input_ftrs, n_classes, layer_size, num_layers, dropout):
        super(Basic_NN, self).__init__()
        self.num_layers = num_layers
        self.base_model = nn.Sequential(nn.Linear(in_features=input_ftrs, out_features=layer_size))
        self.hidden = nn.ModuleList()
        if self.num_layers > 1:
            for k in range(self.num_layers-1):
                self.hidden.append(nn.Sequential(nn.ReLU(), nn.Dropout(p=dropout), nn.Linear(in_features=layer_size, out_features=layer_size)))

        self.output = nn.Sequential(nn.ReLU(), nn.Dropout(p=dropout), nn.Linear(in_features=layer_size, out_features=n_classes))

    def forward(self, x):

        y = self.base_model(x)
        for layer in self.hidden:
            y = layer(y)
        y = self.output(y)

        return y
    
class LeakyReLUMSE(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, pred, actual):
        return torch.mean(F.leaky_relu(actual-pred)**2)


In [ ]:
######################## Curate Correlated Features #############################################################

scaler = StandardScaler()
# The threshold for the AMI scores to decide which features are most correlated with sequence length selection
AMI_correlation_threshold = config['AMI_correlation_threshold']
# The total number of features in the original VPR feature descriptors
total_num_ftrs = coarse_prior_appearance_var.shape[1]

# Scaled training split
appearance_var_train = scaler.fit_transform(coarse_prior_appearance_var[0:train,:])

# Computing the adjusted mutual information correlation between features and sequence length
# in the training set
AMI_score_train = []
for i in range(0, total_num_ftrs, 1):
    AMI_score_train.append(adjusted_mutual_info_score(appearance_var_train[:,i], min_seq_len_for_target[0:train]))

# The indices of the features which have an AMI correlation with sequence length selection which is
# above the selected threshold
curated_ftr_idxs = np.where(np.array(AMI_score_train) > AMI_correlation_threshold)[0].astype(int)

######################### Create Training and Testing Data ##################################################

# Scaled and curated appearance variation feature vectors
curated_appearance_var_train = scaler.fit_transform(coarse_prior_appearance_var[0:train,curated_ftr_idxs])
curated_appearance_var_valid = scaler.transform(coarse_prior_appearance_var[train:cutoff:,curated_ftr_idxs])
curated_appearance_var_test = scaler.transform(coarse_prior_appearance_var[cutoff::,curated_ftr_idxs])

########################## Setup Model Parameters ##########################################################

num_workers = 0
batch_size = config['batch_size']
layer_size = config['layer_size']
num_layers = config['num_layers']
dropout = config['dropout']
lr = config['lr']
max_epoch_number = config['epochs']

device = torch.device('cpu')
num_train_batches = int(np.ceil(train / batch_size))

######################### Load Datasets ###################################################################


train_dataset = TensorDataset(torch.from_numpy(curated_appearance_var_train).float(), torch.from_numpy(min_seq_len_for_target[0:train]))
valid_dataset = TensorDataset(torch.from_numpy(curated_appearance_var_valid).float(), torch.from_numpy(min_seq_len_for_target[train:cutoff]))
tester_dataset = TensorDataset(torch.from_numpy(curated_appearance_var_test).float(), torch.from_numpy(min_seq_len_for_target[cutoff::]))

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True) #, shuffle=True
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, num_workers=num_workers)
test_dataloader = DataLoader(tester_dataset, batch_size=batch_size, num_workers=num_workers)

################################# Initialize Model ###############################################################

model = Basic_NN(curated_ftr_idxs.shape[0], 1, layer_size, num_layers, dropout)
model.train()
model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)

criterion = LeakyReLUMSE()

def round_to_odd(output):
    return 2 * torch.round(output / 2) + 1

################################### Training ########################################################################
print('Starting Training')
# Run training
epoch = 0
iteration = 0
valid_loss_value = 1
patience_counter = 0
old_valid_loss = float('inf')
all_losses = []
all_valid_losses = []
all_median_seqs = []
all_sect_succs = []
while True:
    model_result_all = []
    targets_all = []
    batch_losses = []
    train_filenames = []
    all_preds = []
    for inputs, targets in train_dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        model_result = model(inputs).squeeze()
        loss = criterion(model_result, targets.float().squeeze())
        
        loss.backward()
        optimizer.step()
        

        batch_losses.append(loss.item())
        model_result_all.extend(model_result.detach().cpu().numpy())
        targets_all = targets_all + targets.cpu().numpy().tolist()
        all_preds = all_preds + model_result.detach().cpu().numpy().tolist()

        iteration += 1

    model.eval()
    with torch.no_grad():
        valid_targets_all = []
        valid_all_preds = []
        valid_batch_losses = []
        train_filenames = []
        for inputs, targets in valid_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)

            model_result = model(inputs).squeeze()
            valid_loss = criterion(model_result.squeeze(), targets.float()) 

            valid_batch_losses.append(valid_loss.item())
            valid_targets_all = valid_targets_all + targets.cpu().numpy().tolist()
            valid_all_preds = valid_all_preds + model_result.detach().cpu().numpy().tolist()

    with torch.no_grad():
        test_targets_all = []
        test_all_preds = []
        for inputs, targets in test_dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            model_result = model(inputs).squeeze()
            test_targets_all = test_targets_all + targets.cpu().numpy().tolist()
            try:
                test_all_preds = test_all_preds + model_result.detach().cpu().numpy().tolist()
            except:
                test_all_preds = test_all_preds + [model_result.detach().cpu().numpy().tolist()]
    model.train()

    loss_value = np.mean(batch_losses)
    valid_loss_value = np.mean(valid_batch_losses)
    all_losses.append(loss_value)
    all_valid_losses.append(valid_loss_value)
    # print('Epoch {} loss: {}'.format(epoch, loss_value))
    if valid_loss_value > old_valid_loss:
        patience_counter += 1
    else:
        patience_counter = 0
        old_valid_loss = valid_loss_value

    if patience_counter > 3:
        break

    if max_epoch_number < epoch:
        break
    epoch += 1


# with torch.no_grad():
#     test_targets_all = []
#     test_all_preds = []
#     for inputs, targets in test_dataloader:
#         inputs, targets = inputs.to(device), targets.to(device)
#         model_result = model(inputs).squeeze()
#         test_targets_all = test_targets_all + targets.cpu().numpy().tolist()
#         try:
#             test_all_preds = test_all_preds + model_result.detach().cpu().numpy().tolist()
#         except:
#             test_all_preds = test_all_preds + [model_result.detach().cpu().numpy().tolist()]